In [1]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
import os
import tensorflow as tf
import tensorflow.keras.layers as tfl
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import random

from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, RandomRotation

os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

# Load test dataset

In [3]:
# dataset creation function
def create_dataset_labels(path):
    dirs = os.listdir(path+'Kt')
    classes = []
    labels = []
    masks = []
    images = []
    filenum = 0
    for d in dirs:
        classes.append(d)
        files = os.listdir(path + 'Kt/' + d)
        filenum += len(files)
        for n, f in tqdm(enumerate(files), total=len(files)):
            label = float(f.split('.')[0].split('_')[1])/10000 # depth
            labels.append(label)  # depth as the label
            image = tf.keras.utils.load_img(path + '/Kt/'+ d + '/'+ f)
            image = tf.keras.utils.img_to_array(image)
            images.append(image)
            
            mask = tf.keras.utils.load_img(path + '/mask/'+ d + '/'+ f,color_mode="grayscale")
            mask = tf.keras.utils.img_to_array(mask)
            mask = mask != 0
            
            masks.append(mask)
            
    print('Found '+str(filenum)+' files in '+ str(len(classes)) +' directories.')

    images = tf.convert_to_tensor(images, dtype = tf.float32)
    masks = tf.convert_to_tensor(masks, dtype = tf.bool)
    labels = tf.convert_to_tensor(labels, dtype = tf.float32)

    dataset = tf.data.Dataset.from_tensor_slices((images,masks,labels))
    dataset = dataset.shuffle(buffer_size = 11200, reshuffle_each_iteration = True)
#     dataset = dataset.batch(32)
    return(dataset)

In [4]:
# create test dataset
test_dataset = create_dataset_labels('validation/')
print(test_dataset)

100%|██████████| 1200/1200 [00:01<00:00, 609.02it/s]


Found 6000 files in 2 directories.
<ShuffleDataset shapes: ((128, 128, 3), (128, 128, 1), ()), types: (tf.float32, tf.bool, tf.float32)>


# UNet for vessel segmentation

In [5]:
# tversky function for DSC metrics
from tensorflow.keras import backend as K 
def tversky(y_true, y_pred):
    epsilon = 1e-5
    smooth = 1
    y_true = float(y_true)
    y_true_pos = K.flatten(y_true)
    y_pred_pos = K.flatten(y_pred)
    true_pos = K.sum(y_true_pos * y_pred_pos)
    false_neg = K.sum(y_true_pos * (1-y_pred_pos))
    false_pos = K.sum((1-y_true_pos)*y_pred_pos)
    alpha = 0.5
    return (true_pos + smooth)/(true_pos + alpha*false_neg + (1-alpha)*false_pos + smooth)

In [6]:
# load model 'UNet.h5'
model1 = tf.keras.models.load_model('UNet.h5',custom_objects={'tversky':tversky})

In [7]:
# evaluate model on test_dataset
model1.evaluate(test_dataset.map(lambda x, y, z: (x, y)).batch(32))

188/188 [==============================] - 19s 60ms/step - loss: 0.0700 - tversky: 0.7451 - precision_2: 0.8170


[0.069969043135643, 0.7450715899467468, 0.816999077796936]

# ResNet for depth estimation

In [8]:
# load model 'ResNet.h5'
model2 = tf.keras.models.load_model('ResNet.h5')

In [9]:
# evaluate model on test_dataset
model2.evaluate(test_dataset.map(lambda x, y, z: (x, z)).batch(32))

188/188 [==============================] - 8s 24ms/step - loss: 3.2449e-04 - mae: 0.0128


[0.0003244938561692834, 0.012848706915974617]